In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [38]:
# check for URL patterns
https://www.imdb.com/search/title/?title_type=tv_series&release_date=2012-01-01,2020-12-31&countries=us&languages=en&runtime=5,&view=simple&sort=release_date,asc&count=250&start=9501&ref_=adv_nxt
https://www.imdb.com/search/title/?title_type=tv_series&release_date=2012-01-01,2020-12-31&countries=us&languages=en&runtime=5,&view=simple&sort=release_date,asc&count=250
https://www.imdb.com/search/title/?title_type=tv_series&release_date=2012-01-01,2020-12-31&countries=us&languages=en&runtime=5,&view=simple&sort=release_date,asc&count=250&start=251&ref_=adv_nxt

In [ ]:
# create list for movies with missing info - FINAL VERSION FOR EXTRACTING LINKS
error_movies = []
#create dictionary
data = {
      'title': [],
      'year': [],
      'rating': [], 'link' : [],
    }

#counter
x = 0
base_url = "https://www.imdb.com/search/title/?title_type=tv_series&release_date=2012-01-01,2020-12-31&countries=us&languages=en&runtime=5,&view=simple&sort=release_date,asc&count=250"

while x<=38:
    if x == 0:
        url = base_url
    else:
        url = base_url+"&start={}&ref_=adv_nxt".format(x*250 + 1)
    response = requests.get(url)
    response.status_code
    soup = BeautifulSoup(response.text, 'lxml')
    show_elements=soup.find_all('div', class_='lister-item-content')

    try:
        for element in show_elements:
        #extract title
    
            title = element.find('div', class_='col-title').find('a')
            data['title'].append(title.text)
            
        #extract link
            link = title['href']
            data['link'].append(link)
    
        #extract year
    
            year_released = element.find('div', class_="col-title").find('a').findNext().text
            year_text = year_released.replace('(', '')
            year_text = year_text.replace('–', '')
            year_text = year_text.replace(')', '')
            year_text = year_text.replace('I', '')
            data['year'].append(year_text)
    
        # extract rating
            ratings = element.find('div', class_="col-imdb-rating").find('strong').text.strip()[-3:]
            data['rating'].append(ratings)
    except:
        data['rating'].append('Missing')
        error_movies.append(title.text)
    x+=1
    print(x)

In [40]:
error_movies

In [41]:
df = pd.DataFrame(data)

In [ ]:
df.head()

In [20]:
df.dtypes

In [7]:
df.shape

In [25]:
# save links to csv
df.to_csv('file_name.csv')

In [120]:
import re

def get_show_dict(link):
    
    inner_page = {'title': None, 'year':None, 'rating':None, 'num_reviews': None, 'genre':None,
                  'certificate': None, 'num_episodes':None, 'actors':None, 'network':None, 'runtime':None, 'awards':None}

    '''
    From IMDB link stub, request show html, parse with BeautifulSoup, and
    collect 
        - title 
        - number of reviews
        - runtime 
        - genre
        - certificate
        - number of episodes
        - actors
        - network
    Return information as a dictionary.
    '''
    
    base_url = 'https://www.imdb.com/'
    
    #Create full url to scrape
    url = base_url + link
    
    #Request HTML and parse
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    
    # get title of show
    title = soup.find('div', class_='title-overview').find('div', class_='title_wrapper').find('h1').text.strip()
    inner_page['title']=title

    try:
        # YEAR ON SHOW PAGE
        year = soup.find('div', class_='title_wrapper').find('div', class_='subtext').find_all('a')[-1].text.split()[-1]
        clean_year = year.replace(')', '').replace('(', '')
        inner_page['year'] =clean_year
    except:
        inner_page['year'] ='Missing'
        
    try:
        # RATING ON SHOW PAGE

        rating = soup.find('div', class_='ratingValue').find('span', itemprop='ratingValue').text
        inner_page['rating'] = rating
    except:
        inner_page['rating']= 'Missing'
        
        # number of reviews
    try:
        num_reviews = soup.find('span', class_='small').text
        inner_page['num_reviews'] = num_reviews
    except:
        inner_page['num_reviews'] = 0

    # genres & certificate

    try:
        for genre in soup.find('div', {'id':'titleStoryLine'}).find_all('div', class_='see-more inline canwrap')[1].find_all('a'):
            clean_genre = genre.text.strip()
            inner_page['genre'] = clean_genre
    except:
        inner_page['genre'] = 'Missing'

    try:
        certificate = soup.find('div', {'id':'titleStoryLine'}).find('div', class_='txt-block').find('span').text
        inner_page['certificate'] = certificate
    except:
        inner_page['certificate'] = 'Missing'

    # num of episodes

    try:
        episodes = soup.find('div', class_='bp_content').find('span').text.replace('episodes', '').strip()
        inner_page['num_episodes'] = episodes
    except:
        inner_page['num_episodes'] = 0

    #cast

    try:
        for actor in soup.find_all('div', class_="credit_summary_item")[-1].find_all('a')[:-1]:
            clean_actor = actor.text.strip()
            inner_page['actors'] = clean_actor
    except:
        inner_page['actors'] = 'Missing'

    #network

    try:
        for network in soup.find('div', {'id':'titleDetails'}).find_all('div', class_='txt-block')[5].find_all('a')[:-1]:
            clean_network = network.text.strip()
            inner_page['network'] = clean_network
    except:
        inner_page['network'] = 'Missing'

    # runtime
    try:
        runtime = soup.find('div', {'id':'titleDetails'}).find('time').text
        inner_page['runtime'] = runtime
    except:
        inner_page['runtime'] = 'Missing'

    if soup.findAll(text=re.compile('award')):
        inner_page['awards'] = 1
    else:
        inner_page['awards'] = 0

    return inner_page

# this works, got rid of the brackets in dataframe

In [223]:
# creating test list to make sure everything is working
test_list = []

for link in links_df[:100].link:
    test_list.append(get_show_dict(link))

In [229]:
len(test_list)

In [52]:
test_df = pd.DataFrame(test_list)
test_df.head(50)

In [22]:
full_set = []

for link in links_df.link:
    full_set.append(get_show_dict(link))

In [23]:
full = pd.DataFrame(full_set)

In [12]:
full.head()

In [17]:
full.shape

In [17]:
merged = pd.merge(links_df, full, how='outer', left_on = 'title', right_on='title')

In [ ]:
merged.head()

In [ ]:
merged.drop(columns=['Unnamed: 0', 'link'], inplace=True)

In [ ]:
merged.head()

In [ ]:
merged.to_csv('merged_with_links_2.csv')